In [1]:
import cv2
from ultralytics import YOLO

In [3]:

# Carga del modelo YOLO
model_car = YOLO('yolov8n.pt')

# Ruta al archivo de video
video_path = 'sample2.mp4'

# Inicia la captura de video desde el archivo MP4
cap = cv2.VideoCapture(video_path)

while True:
    # Lee el siguiente frame del video
    ret, frame = cap.read()

    # Si no se puede leer el frame, sal del bucle
    if not ret:
        break

    # Detecta coches en el frame
    results_cars = model_car(frame)

    # Para cada detección de coche
    for r_car in results_cars:
        boxes_car = r_car.boxes

        for box_car in boxes_car:
            if int(box_car.cls[0]) == 2:
                # Coordenadas del cuadro del coche
                x1_car, y1_car, x2_car, y2_car = box_car.xyxy[0]

                # Definir la región inferior para buscar matrículas
                roi_bottom = frame[int((y1_car + y2_car) / 2):int(y2_car), int(x1_car):int(x2_car)]

                # Convertir la región a escala de grises
                gray_roi_bottom = cv2.cvtColor(roi_bottom, cv2.COLOR_BGR2GRAY)

                # Proceso de detección y reconocimiento de matrículas aquí (usando el código que ya tienes)
                edges = cv2.Canny(roi_bottom, 30, 200)

                cnts, new = cv2.findContours(edges.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

                cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:30]

                plate = None
                for c in cnts:
                    perimeter = cv2.arcLength(c, True)
                    edges_count = cv2.approxPolyDP(c, 0.02 * perimeter, True)
                    if len(edges_count) == 4:
                        x, y, w, h = cv2.boundingRect(c)
                        x, y, w, h = int(x + x1_car), int(y + (y1_car + y2_car) / 2), int(w), int(h)
                        plate = frame[y:y + h, x + 10:x + w]

                # Dibuja el contenedor y clase del coche
                cv2.rectangle(frame, (int(x1_car), int(y1_car)), (int(x2_car), int(y2_car)), (0, 0, 255), 2)
                cv2.putText(frame, "Car", [int(x1_car), int(y1_car)], cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                # Muestra la placa después de encontrar la mejor candidata
                if plate is not None and plate.shape[0] > 0 and plate.shape[1] > 0:
                    cv2.imshow('Plate', plate)

    # Muestra el frame con las detecciones
    cv2.imshow('Video con Detección', frame)

    # Sale del bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libera la captura de video y cierra todas las ventanas
cap.release()
cv2.destroyAllWindows()



0: 640x640 6 cars, 1 truck, 114.5ms
Speed: 6.5ms preprocess, 114.5ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 cars, 1 bus, 1 truck, 150.9ms
Speed: 8.0ms preprocess, 150.9ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 cars, 1 bus, 1 truck, 128.3ms
Speed: 7.5ms preprocess, 128.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 8 cars, 1 truck, 138.2ms
Speed: 7.5ms preprocess, 138.2ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 cars, 1 truck, 127.7ms
Speed: 6.2ms preprocess, 127.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 cars, 115.7ms
Speed: 6.0ms preprocess, 115.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 cars, 1 truck, 114.8ms
Speed: 6.0ms preprocess, 114.8ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 6 cars, 1 truck, 123.6ms
Speed: 4.0ms 